# 주의사항
1. windows 환경에서는 빌드된 ffmpeg 실행파일을 환경 변수 > Path 에 등록해야 한다.
2. gym 0.18.0 에는 video로 저장할 수 없는 버그가 있다. [refer](https://github.com/openai/gym/issues/1925)
3. wrappers.Monitor의 video_callable 파라미터로 녹화할 에피소드를 지정할 수 있다. capture_frame이 호출되는 시점은 _after_step이다.  

* Monitor wrapper에서 반환된 env는 step을 실행할 때 _before_step(action), env.step(action), _after_step(action) 순으로 실행 후 return observatoin, reward ... 한다.  
* 프레임 캡쳐는 위와 같이 매 스탭 진행된다. 
* recorder를 종료하고 재시작하는 것은 reset_video_recorder에서 담당한다.
  * 먼저, 열려있는 recorder를 종료한다.
  * 이번 에피소드가 녹화할 에피소드라면 파일을 생성한다.
  * 학습을 진행하면 파일에 프레임이 저장된다.
* n_epi를 녹화하려면 n_epi + 1이 시작된 이후, 또는 안전하게 n_epi + 2까지 가서 녹화해야 한다.

In [1]:
from actor_critic import *
import torch    # pytorch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [2]:
import gym
from IPython.display import Video

class Recorder:
    def __init__(self, env):
        self.n_epi_set = set()
        self.env = gym.wrappers.Monitor(env, "./gym-results", force=True, video_callable=self.save_or_not)

    def update(self, n_epi_li):
        if hasattr(n_epi_li, '__iter__') and all(map(lambda x : type(x) == type(int()), n_epi_li)):
            self.n_epi_set.update(n_epi_li)
        else:
            raise Exception("Recorder에 잘못된 n_epi가 들어옴")

    def wrapped_env(self):
        return self.env
        
    def save_or_not(self, n_epi):
        return n_epi in self.n_epi_set

    def replay(self):
        if len(self.n_epi_set) == 0:
            print('조건을 만족한 episode가 없음.')
        else:
            print(self.n_epi_set)
            last_epi = list(self.n_epi_set)[-1]
            video_url = f'./gym-results/openaigym.video.{self.env.file_infix}.video{last_epi-1:0>6}.mp4'
            Video(video_url)

In [3]:
def main():
    recorder = Recorder(gym.make('CartPole-v1'))
    env = recorder.wrapped_env()
    # policy 대신 model class를 만들었다. 
    # actor-critic 둘 다 있어야 하기 때문에 policy와 value를 둘 다 얻을 수 있게 함.
    model = ActorCritic()    
    print_interval = 20
    score = 0.0

    for n_epi in range(10000):
        done = False
        s = env.reset()
        while not done:
            for t in range(n_rollout):
                # 확률 분포 구하고
                prob = model.pi(torch.from_numpy(s).float())
                # 확률 분포 모델 만들고
                m = Categorical(prob)
                # 샘플링하고
                a = m.sample().item()
                # 환경에 넘겨주고 다음 observation 얻고
                s_prime, r, done, info = env.step(a)
                # TD의 경우 매번 학습이 가능하지만 모아서 batch learning 하니까 학습이 더 잘 됐다.
                model.put_data((s,a,r,s_prime,done))
                
                s = s_prime
                score += r
                
                if done:
                    break                     
            
            model.train_net()

        if score/print_interval > 400:
            recorder.update([n_epi + 1])
            if len(recorder.n_epi_set) >= 5:
                break

        if n_epi%print_interval==0 and n_epi!=0:
            print("# of episode :{}, avg score : {:.1f}".format(n_epi, score/print_interval))
            score = 0.0
    env.close()
    
    recorder.replay()

In [4]:
if __name__ == '__main__':
    main()

# of episode :20, avg score : 24.3
# of episode :40, avg score : 21.5
# of episode :60, avg score : 18.6
# of episode :80, avg score : 17.2
# of episode :100, avg score : 15.6
# of episode :120, avg score : 16.6
# of episode :140, avg score : 19.9
# of episode :160, avg score : 22.4
# of episode :180, avg score : 27.1
조건을 만족한 episode가 없음.
